In [2]:
from __future__ import print_function
import glob
import numpy as np
import math
from pyhdf.SD import SD, SDC
import matplotlib
#matplotlib.use('TkAgg')
import pylab
import matplotlib.pyplot as plt
import datetime

import subprocess
from numpy import genfromtxt
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#import plotly.graph_objs as go
from scipy.spatial import cKDTree
import bs4
import itertools
import bottleneck as bn
from scipy.signal import lfilter
from itertools import compress
from numpy import inf
import pandas as pd

In [3]:
#DATA = np.load('/home/kumarv/tayal007/MODIS_TILES/h08v05.npy')
#load ndvi data from 2013
DATA = np.load('/Users/kshitij/Downloads/data2013.npy')
#load mask from landsat product
mask = np.load('/Users/kshitij/Downloads/mask.npy')

In [5]:
def readModisfile(path):
    global mask
    file = SD(path, SDC.READ)
    sds_obj = file.select('Land_Cover_Type_1') 
    mask = np.array(sds_obj.get())
    return mask
def percentage(part, shape):
    whole = shape[0] * shape[1] * shape[2]
    return 100 * float(part)/float(whole)

def cleanNDVI(NDVISTACK):
	print ("Missing value in raw data is ", np.count_nonzero(np.isnan(NDVISTACK)),'which is ' ,percentage(np.count_nonzero(np.isnan(NDVISTACK)),NDVISTACK.shape),'%')
	print ("Infinity value in raw data is ", np.count_nonzero(np.isinf(NDVISTACK)),'which is ' ,percentage(np.count_nonzero(np.isinf(NDVISTACK)),NDVISTACK.shape),'%')
	print ("Greater than 1 is ", np.count_nonzero(NDVISTACK > 1),'which is ' ,percentage(np.count_nonzero(NDVISTACK > 1),NDVISTACK.shape),'%')
	print ("Less than -1 is ", np.count_nonzero(NDVISTACK < -1),'which is ' ,percentage(np.count_nonzero(NDVISTACK < -1),NDVISTACK.shape),'%')
	
	CLEANED_DATA = np.copy(NDVISTACK)
	CLEANED_DATA[NDVISTACK == inf] = np.nan
	CLEANED_DATA[NDVISTACK == -inf] = np.nan
	CLEANED_DATA[NDVISTACK > 1] = np.nan
	CLEANED_DATA[NDVISTACK < -1] = np.nan

	print ("After Cleaning Missing value is ", np.count_nonzero(np.isnan(CLEANED_DATA)),'which is ' ,percentage(np.count_nonzero(np.isnan(CLEANED_DATA)),NDVISTACK.shape),'%')
	return CLEANED_DATA;
#data2013 = np.copy(DATA[:,:,586:631])
data2013 = DATA
data2013[data2013 < 0] = 0
data2013 = cleanNDVI(data2013)
#data2013 = data2013.reshape(data2013.shape[0] * data2013.shape[1], data2013.shape[2])
mask = readModisfile('/panfs/roc/groups/6/kumarv/tayal007/MODIS_yearly/MCD12Q1.A2013001.h08v05.051.2014308185137.hdf')
crop_data = data2013[mask==12]

/Users/kshitij/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in less


Missing value in raw data is  16338907 which is  6.3035906635802466 %
Infinity value in raw data is  160282 which is  0.06183719135802469 %


/Users/kshitij/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in greater
  


Greater than 1 is  1946976 which is  0.7511481481481481 %


/Users/kshitij/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in less
  from ipykernel import kernelapp as app


Less than -1 is  0 which is  0.0 %


/Users/kshitij/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in greater
/Users/kshitij/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in less


After Cleaning Missing value is  18285883 which is  7.054738811728395 %


HDF4Error: SD: no such file

In [4]:
row_mean = np.nanmean(crop_data, axis=1)
inds = np.where(np.isnan(crop_data))
crop_data[inds] = np.take(row_mean, inds[0])

In [5]:
non_crop_data = data2013[mask!=12]
row_mean = np.nanmean(non_crop_data, axis=1)
inds = np.where(np.isnan(non_crop_data))
non_crop_data[inds] = np.take(row_mean, inds[0])
non_crop_data = non_crop_data[~np.isnan(non_crop_data).any(axis=1)]

/home/kumarv/tayal007/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: RuntimeWarning:

Mean of empty slice



In [6]:
crop_data = crop_data[~np.isnan(crop_data).any(axis=1)]
print (crop_data.shape)
print (non_crop_data.shape)

(229337, 45)
(5530662, 45)


In [43]:
crop_data = np.load("AUT_CROP_Encoding_8.npy")
non_crop_data = np.load("AUT_NON_CROP_Encoding_8.npy")

In [44]:
crop_y = np.ones((229337,1))
non_crop_y = np.zeros((5530662,1))

In [45]:
X = np.r_[crop_data,non_crop_data]
Y = np.r_[crop_y,non_crop_y]

In [46]:
from sklearn.model_selection import StratifiedShuffleSplit
sss= StratifiedShuffleSplit(n_splits=1, test_size=0.6, random_state=0)
for train_index, test_index in sss.split(X, Y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]

In [40]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100,n_jobs=25,verbose=1 ,max_depth=20, random_state=0)
clf.fit(X_train, y_train.ravel())

[Parallel(n_jobs=25)]: Done 100 out of 100 | elapsed:  1.0min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=25,
            oob_score=False, random_state=0, verbose=1, warm_start=False)

In [41]:
y_pred = clf.predict(X_test)

[Parallel(n_jobs=25)]: Done 100 out of 100 | elapsed:    7.2s finished


In [31]:
y_pred.shape

(3456000,)

[[3318372      26]
 [ 137596       6]]
             precision    recall  f1-score   support

        0.0       0.96      1.00      0.98   3318398
        1.0       0.19      0.00      0.00    137602

avg / total       0.93      0.96      0.94   3456000



In [14]:
def RandomForest(data,labels):
    
    sss= StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0) #Stratified Sampling

    for train_index, test_index in sss.split(X, Y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        
    clf = RandomForestClassifier(n_estimators=100,n_jobs=25,verbose=1 ,max_depth=20, random_state=0)
    clf.fit(X_train, y_train.ravel())
    y_pred = clf.predict(X_test)
    
    print(confusion_matrix(y_test.ravel(), y_pred))  
    print(classification_report(y_test.ravel(), y_pred))  

0

In [15]:
y_pred[y_pred == 0].size

3456000

In [16]:
y_test[y_test == 1].size

137602

In [17]:
y_test[y_test == 0].size

3318398